# Boxplots aus Realtimearchiv
- Stand 09.01.2024

In [1]:
import duckdb
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.ticker as plticker
import matplotlib.patheffects as pe
import seaborn as sns
import datetime as dt

In [18]:
pd.options.display.max_columns = 500

## Funktionen

### Anpassen der Beschriftung 

In [19]:
def label_short(hstname):
    s = hstname.partition(' ') #teilt nach dem ersten Auftreten des Separators auf
    if s[2] == '': 
        name_short = s[0][0:8] 
    else:
        name_short = s[2][0:8]
    return name_short


In [20]:
#test
label_short('Oldenburg Am Spreeken')

'Am Spree'

## Abruf der Daten aus DB und Erstellen DF

In [21]:
duck = duckdb.connect('db/rt_dev.db')

In [22]:
duck.sql("SHOW TABLES")

┌────────────────────┐
│        name        │
│      varchar       │
├────────────────────┤
│ fahrten_2024_01_08 │
│ fahrten_2024_01_10 │
│ lin_dm             │
│ verlauf_2024_01_08 │
│ verlauf_2024_01_10 │
│ zusatz_2024_01_08  │
│ zusatz_2024_01_10  │
└────────────────────┘

In [23]:
duck.sql("DESCRIBE fahrten_2024_01_10")

┌───────────────────────┬─────────────┬─────────┬─────────┬─────────┬───────┐
│      column_name      │ column_type │  null   │   key   │ default │ extra │
│        varchar        │   varchar   │ varchar │ varchar │ varchar │ int32 │
├───────────────────────┼─────────────┼─────────┼─────────┼─────────┼───────┤
│ datum                 │ VARCHAR     │ YES     │ NULL    │ NULL    │  NULL │
│ fnr                   │ VARCHAR     │ YES     │ NULL    │ NULL    │  NULL │
│ destination           │ VARCHAR     │ YES     │ NULL    │ NULL    │  NULL │
│ hasRealtime           │ BOOLEAN     │ YES     │ NULL    │ NULL    │  NULL │
│ vu                    │ VARCHAR     │ YES     │ NULL    │ NULL    │  NULL │
│ lineid                │ VARCHAR     │ YES     │ NULL    │ NULL    │  NULL │
│ lineid_short          │ VARCHAR     │ YES     │ NULL    │ NULL    │  NULL │
│ lineshort             │ VARCHAR     │ YES     │ NULL    │ NULL    │  NULL │
│ reported_cancelled    │ BOOLEAN     │ YES     │ NULL    │ NULL

In [24]:
df_verlauf = duck.sql("SELECT * FROM verlauf_2024_01_10").df()

In [25]:
df_fahrten = duck.sql("SELECT * FROM fahrten_2024_01_10").df()

In [26]:
duck.close()

In [27]:
df_fahrten.head(2)

,datum,fnr,destination,hasRealtime,vu,lineid,lineid_short,lineshort,reported_cancelled,journey_cancelled,ts_reported_cancelled,cancelled_kum,deviceid,clientid,journeyrttype,fahrtstarttime,fahrtstartstationname,fahrtstartstationdhid,fahrtendtime,fahrtendstationname,fahrtendstationdhid
0,2023-10-26,154,Borchersweg,True,Verkehr und Wasser GmbH (VWG),de:VBN:302:,de:VBN:302,302,True,True,2023-10-26T09:06:07.068+02:00,True,1026-154#!ADD!#vwg#,vwg,SCHEDULED,2023-10-26T08:40:00+02:00,Oldenburg(Oldb) ZOB,de:03403:27022:1:2,2023-10-26T09:06:00+02:00,Oldenburg(Oldb) Borchersweg,de:03403:12470:1:2
1,2023-10-26,383,Drielake,True,Verkehr und Wasser GmbH (VWG),de:VBN:330:1,de:VBN:330,330,False,False,,False,1026-383#!ADD!#vwg#,vwg,SCHEDULED,2023-10-26T08:18:00+02:00,Wiefelstede Schulzentrum,de:03451:99166::2,2023-10-26T09:08:00+02:00,Oldenburg(Oldb) Drielake,de:03403:12499:1:1


In [28]:
df_verlauf[df_verlauf.lineshortname == '910'].head(3)

,operday,journeyOperator,deviceid,lineshortname,ex_lineid,fnr,index,has_rt,dschedtime,aschedtime,dep_del,arr_del,station_nr,station_name,lat,lon,canc,additional,ts_reported_cancelled,reported_cancelled
143,2023-10-26,SBV Janßen GmbH & Co. KG,1026-25910003#!ADD!#janssen#,910,de:VBN-VGC:910:,3,1,True,2023-10-26 08:35:00,NaT,0.0,NaN,105310,Friesoythe Hansaplatz,53.022450,7.861714,False,False,,False
144,2023-10-26,SBV Janßen GmbH & Co. KG,1026-25910003#!ADD!#janssen#,910,de:VBN-VGC:910:,3,2,True,2023-10-26 08:36:00,2023-10-26 08:36:00,0.0,0.0,9904098,Friesoythe Ev. Kirche,53.025111,7.867728,False,False,,False
145,2023-10-26,SBV Janßen GmbH & Co. KG,1026-25910003#!ADD!#janssen#,910,de:VBN-VGC:910:,3,3,True,2023-10-26 08:37:00,2023-10-26 08:37:00,0.0,0.0,9904245,Altenoythe ehemalige Schule,53.031736,7.878119,False,False,,False


In [52]:
df_fahrten.lineshort.drop_duplicates()

0     302
1     330
3     340
4     910
10    680
Name: lineshort, dtype: object

In [53]:
#Ermitteln Anteil Echtzeit
auswahl = '680'
f'Anteil Echtzeit Linie {auswahl}: {round(df_fahrten[(df_fahrten.lineshort == auswahl) & (df_fahrten.hasRealtime == True)].shape[0] / df_fahrten[(df_fahrten.lineshort == auswahl) ].shape[0], 2) * 100}%'

'Anteil Echtzeit Linie 680: 68.0%'

## Ermitteln der Fahrtnummer und Überprüfung anhand Fahrtstartende

In [54]:
df_fahrten.dtypes

datum                    object
fnr                      object
destination              object
hasRealtime                bool
vu                       object
lineid                   object
lineid_short             object
lineshort                object
reported_cancelled         bool
journey_cancelled        object
ts_reported_cancelled    object
cancelled_kum              bool
deviceid                 object
clientid                 object
journeyrttype            object
fahrtstarttime           object
fahrtstartstationname    object
fahrtstartstationdhid    object
fahrtendtime             object
fahrtendstationname      object
fahrtendstationdhid      object
dtype: object

In [55]:
df_fahrten.fahrtstarttime.dt.hour

AttributeError: Can only use .dt accessor with datetimelike values

## Auswahl der Linie und Ermitteln Fahrtliste

In [56]:
auswahl_linie = '680'
start = '2023-11-01'
ende = '2024-12-31'
df_fnr = df_verlauf[(df_verlauf.lineshortname == auswahl_linie) & (df_verlauf.operday >= start) & (df_verlauf.operday <= ende)]['fnr'].drop_duplicates().sort_values()

In [76]:
for i, fnr in df_fnr[0:1].items():
    print(fnr)

1680001


In [69]:
fnr = '1680001'
df_sel = df_verlauf[(df_verlauf.fnr == fnr) & ~(df_verlauf.dep_del.isnull()) & (df_verlauf.dep_del < 30) \
                    & (df_verlauf.operday >= start) & (df_verlauf.operday <= ende)]\
                        [['fnr','index','operday','station_name' ,'dep_del']].sort_values(['operday', 'fnr', 'index'])

In [86]:
df_verlauf[(df_verlauf.fnr == fnr) & (df_verlauf.lineshortname == auswahl_linie) & ~(df_verlauf.dep_del.isnull()) & (df_verlauf.dep_del < 30) \
                        & (df_verlauf.operday >= start) & (df_verlauf.operday <= ende)][['index','station_name']].groupby('index').agg({'station_name': 'last'}).reset_index()

,index,station_name
0,1,Bremen Gröpelingen
1,2,Bremen DIAKO Bremen
2,3,Bremen Ritterhuder Heerstraße
3,4,Bremen Mittelsbürener Landstraße
4,5,Bremen Bahnhof Burg
5,6,Bremen Stader Landstraße
6,7,Bremen Stockholmer Straße
7,8,Ritterhude Struckbergstraße
8,9,Ritterhude Schillerstraße
9,10,Ritterhude An der Untermühle


In [88]:
for i, fnr in df_fnr[0:].items():
    print(fnr)
    df_sel = df_verlauf[(df_verlauf.fnr == fnr) & (df_verlauf.lineshortname == auswahl_linie) & ~(df_verlauf.dep_del.isnull()) & (df_verlauf.dep_del < 30) \
                        & (df_verlauf.operday >= start) & (df_verlauf.operday <= ende)][['index','operday','dep_del', 'dschedtime']]
    df_sel_label = df_verlauf[(df_verlauf.fnr == fnr) & (df_verlauf.lineshortname == auswahl_linie) & ~(df_verlauf.dep_del.isnull()) & (df_verlauf.dep_del < 30) \
                        & (df_verlauf.operday >= start) & (df_verlauf.operday <= ende)][['index','station_name']].groupby('index').agg({'station_name': 'first'}).reset_index()

    anzahl_fahrten = len(df_sel.operday.drop_duplicates())
    min_day = df_sel.operday.min()
    max_day = df_sel.operday.max()
    
    df_sel_label['x_label'] = df_sel_label.apply(lambda x: str(x['index']).zfill(2) + ' ' + label_short(x['station_name']), axis = 1)
    fig, ax = plt.subplots(1,1, figsize=(30, 20))
    flierprops = dict(marker='o', markerfacecolor='lightgray', markersize=2, linestyle='none')
    boxprops = dict(facecolor=(.4, .6, .8, .5))
    medianprops = dict(linestyle='-.', linewidth=2.5, color='firebrick')

    sns.boxplot(ax = ax, data=df_sel, x='index', y='dep_del', boxprops= boxprops, notch=False, medianprops =  medianprops, flierprops=flierprops)

    df_label_x = df_sel_label[['index', 'x_label']].drop_duplicates().sort_values('index')
    df_label_x['index'] = df_label_x.apply(lambda x: x['index'] - 1, axis=1)

    ax.set_xlabel('Haltestellenindex')
    ax.set_ylabel('Abweichung in Minuten')
    ax.grid(visible = True, axis = 'y', which='major', linewidth = 2)
    ax.grid(visible = True, axis = 'y', which='minor', linewidth = 1)
    ax.set_xticks(ticks=df_label_x['index'], labels=df_label_x['x_label'], rotation=90)
    ax.set_title(f'Fahrtnummer {fnr} Anzahl Fahrten {anzahl_fahrten} von {min_day:%d-%m-%Y} bis {max_day:%d-%m-%Y}', fontsize=16)
    plt.figtext(s= 'Quelle: Realtimearchiv VBN 2024/erstellt:' + dt.datetime.now().strftime('%Y-%m-%d %H:%M'), y=0.02, x=0.98, fontsize=12, ha= 'right' )
    plt.subplots_adjust(top=0.96, bottom= 0.12) #Abstand Plots nach oben und unten
    plt.figtext(s= 'Erläuterung zum Boxplot: Die Box gibt den Bereich an in dem 50% der Fahrten liegen. Der mittlere Strich gibt den Median an, und teilt die Box in 25% \
    die über dem Median liegen und 25% darunter. Striche nach oben sind \
    maximale 1,5 mal so lang wie die Box, gehen jedoch nie über das Minimum/Maximum hinaus. Ausreißer werden gesondert dargestellt. Weiteres https://de.wikipedia.org/wiki/Box-Plot',\
    y=0.03, x=0.02, fontsize=13, ha= 'left', wrap=True)
    plt.savefig(f'plt/{auswahl_linie}_{fnr}.pdf')
    plt.close()


1680001
1680002
1680003
1680004
1680005
1680006
1680007
1680008
1680009
1680010
1680011
1680012
1680013
1680014
1680015
1680016
1680017
1680018
1680019
1680020
1680021
1680022
1680023
1680024
1680025
1680026
1680027
1680028
1680029
1680030
1680031
1680032
1680033
1680034
1680035
1680036
1680037
1680038
1680039
1680040
1680041
1680042
1680043
1680044
1680045
1680046
1680047
1680048
1680049
1680050
1680051
1680052
1680053
1680054
1680055
1680056
1680057
1680058
1680059
1680060
1680061
1680062
1680063
1680064
1680065
1680066
1680067
1680068
1680069
1680070
1680071
1680072
1680073
1680074
1680075
1680076
1680077
1680078
1680302
1680305
1680306
1680307
1680308
1680309
1680310
1680311
1680316
1680317
1680318
1680319
1680320
1680321
1680322
1680323
1680325


In [83]:
df_sel[['index', 'station_name']].drop_duplicates()

,index,station_name
84993,1,Bremen Gröpelingen
84994,2,Bremen DIAKO Bremen
84995,3,Bremen Ritterhuder Heerstraße
84996,4,Bremen Mittelsbürener Landstraße
84997,5,Bremen Bahnhof Burg
84998,6,Bremen Stader Landstraße
84999,7,Bremen Stockholmer Straße
85000,8,Ritterhude Struckbergstraße
85001,9,Ritterhude Schillerstraße
85002,10,Ritterhude An der Untermühle
